In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
from joblib import Parallel, delayed

# lines 6 - 29 from: https://www.danli.org/2021/02/03/single-cell-data-analysis-using-scanpy/ 
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

adata = sc.read_10x_mtx(
    "datasets", 
    var_names="gene_symbols",
    cache=True
)

adata.var_names_make_unique()

#filter 20 highest expressed genes
#sc.pl.highest_expr_genes(adata, n_top=20, )

#filtering out the cells with low gene expression/genes that don't show up in many cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata_filter1 = adata.copy()
adata_filter1.write('adata_first_filter.h5ad')

#annotate mitochondrial genes as 'mt' and calculate qc metrics based on 
adata_filter1.var['mt'] = adata_filter1.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
adata_filter1.var['ribo'] = adata_filter1.var_names.str.startswith(("RPS", "RPL"))
adata_filter1.var['hb'] = adata_filter1.var_names.str.startswith("^HB[^(P)]") #regular expression that chooses all leters after HB besides the capital letter P
#regex is used because pseudogenes have names like HBP1, HBP2, etc 

sc.pp.calculate_qc_metrics(adata_filter1, qc_vars=["mt", "ribo", "hb"], percent_top=None, log1p=False, inplace=True)

In [ ]:
#SKIP
sc.pl.violin(adata_filter1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(adata_filter1, "total_counts", "n_genes_by_counts", color="pct_counts_mt")
sc.pl.scatter(adata_filter1, 'total_counts', 'pct_counts_mt')

In [ ]:
adata_cp1 = adata_filter1[
    (adata_filter1.obs['pct_counts_mt'] < 10) &
    (adata_filter1.obs['pct_counts_ribo'] < 20) &
    (adata_filter1.obs['pct_counts_hb'] < 5),
    :
].copy()

In [ ]:
#SKIP
sc.pl.violin(adata_cp1, ['pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb'], jitter=0.4, multi_panel=True)

In [ ]:
adata_cp2 = adata_cp1.copy()
adata_cp2.obs["doublet_score"] = np.nan
adata_cp2.obs["predicted_doublet"] = np.nan

sample_indices = np.arange(adata_cp1.n_obs)

chunk_size = 20000
num_chunks = adata_cp2.shape[0] // chunk_size
try:
    for i in range(num_chunks+1):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, adata_cp2.shape[0])
        adata_subset = adata_cp2[start:end].copy()
        sc.pp.scrublet(adata_subset)
        adata_cp2.obs.loc[adata_subset.obs.index, "doublet_score"] = adata_subset.obs["doublet_score"]
        adata_cp2.obs.loc[adata_subset.obs.index, "predicted_doublet"] = adata_subset.obs["predicted_doublet"]
        print(f"Scrublet round {i} complete")
        
except MemoryError as e:
    print(f"Not enough memory")
    sys.exit(1)

In [ ]:
adata_cp3 = adata_cp2.copy()
sc.pl.violin(adata_cp2, 'doublet_score')

In [ ]:
threshold = 0.18
adata_singlets = adata_cp3[adata_cp3.obs['doublet_score'] < threshold, :].copy()

In [ ]:
# Saving count data
adata_singlets.layers["counts"] = adata_singlets.X.copy()
# Normalizing to median total counts
sc.pp.normalize_total(adata_singlets)
# Logarithmize the data
sc.pp.log1p(adata_singlets)